In [1]:
import os
import sys
import pathlib
import getpass

In [2]:
# Detect if running in Google Colab environment.
# If so, then clone/install package from GitHub.
# Otherwise, use locally.
try:
    import google.colab
    IN_COLAB = True
    print('IN COLAB')
    
    config_path = pathlib.Path('./config.json')
    if config_path.exists():
        import json
        with open(config_path, 'r') as f:
            j = json.load(f)
        os.environ['GITHUB_TOKEN'] = j['GITHUB_TOKEN']

    else:

        # Request GitHub access token.
        if os.getenv('GITHUB_TOKEN',None) is None:
            os.environ['GITHUB_TOKEN'] = getpass.getpass('GitHub Token: ')
        else:
            print('Using cached GitHub Token')
        
        import json
        with open(config_path, 'w+') as f:
            json.dump(
                {'GITHUB_TOKEN': os.environ['GITHUB_TOKEN']}, 
                f,
            )


        

    # Clone or update repo.
    repo = "makassar-ml"
    repo_url = f"https://{os.environ['GITHUB_TOKEN']}@github.com/news-vt/{repo}.git"
    repo_path = f"/content/{repo}"
    repo_branch = "develop"
    ![ -d $repo_path ] && git -C $repo_path pull || git clone --branch $repo_branch $repo_url
    # !git clone --branch $repo_branch $repo_url

    # Install repo to ensure dependencies are resolved.
    !pip install --upgrade $repo_path

    # Add package location to path.
    sys.path.insert(0, repo_path)

    # Set dataset root path.
    dataset_root = pathlib.Path('./dataset')

except:
    IN_COLAB = False
    dataset_root = pathlib.Path('../datasets/')
    print('NOT IN COLAB')

NOT IN COLAB


In [3]:
# Install future annotations for <3.7
if sys.version_info < (3,7):
    !pip install future-annotations

In [4]:
from __future__ import annotations
import makassar_ml as ml
import pytorch_lightning as pl
import torch
from typing import Optional

In [5]:
class BeijingPM25LightningDataModule(pl.LightningDataModule):
    def __init__(self, 
        root: str, 
        feature_cols: list[int], 
        target_cols: list[int], 
        history: int, 
        horizon: int, 
        split: float,
        batch_size: int,
        ):
        self.root = root
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        self.history = history
        self.horizon = horizon
        self.split = split
        self.batch_size = batch_size

    def prepare_data(self):
        # Download the dataset.
        ml.datasets.BeijingPM25Dataset(
            root=self.root,
            download=True,
            )

    def setup(self, stage: Optional[str] = None):

        # Create train/val datasets for dataloaders.
        if stage == 'fit' or stage is None:
            dataset_train_full = ml.datasets.BeijingPM25Dataset(
                root=self.root,
                download=False,
                train=True,
                split=self.split,
                )
            train_n = len(dataset_train_full)
            train_val_cutoff = train_n - round(train_n*.25) # 75% train, 25% val

            self.dataset_train = torch.utils.data.Subset(dataset_train_full, list(range(0, train_val_cutoff)))
            self.dataset_val = torch.utils.data.Subset(dataset_train_full, list(range(train_val_cutoff, train_n)))

            self.dataset_train_wrap = ml.datasets.TimeseriesForecastDatasetWrapper(
                dataset=self.dataset_train,
                feature_cols=self.feature_cols,
                target_cols=self.target_cols,
                history=self.history,
                horizon=self.horizon,
                )
            self.dataset_val_wrap = ml.datasets.TimeseriesForecastDatasetWrapper(
                dataset=self.dataset_val,
                feature_cols=self.feature_cols,
                target_cols=self.target_cols,
                history=self.history,
                horizon=self.horizon,
                )

        # Create test dataset for dataloaders.
        if stage == 'test' or stage is None:
            self.dataset_test = ml.datasets.BeijingPM25Dataset(
                root=self.root,
                download=False,
                train=False,
                split=self.split,
                )
            self.dataset_test_wrap = ml.datasets.TimeseriesForecastDatasetWrapper(
                dataset=self.dataset_test,
                feature_cols=self.feature_cols,
                target_cols=self.target_cols,
                history=self.history,
                horizon=self.horizon,
                )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            dataset=self.dataset_train_wrap,
            batch_size=self.batch_size,
            )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            dataset=self.dataset_val_wrap,
            batch_size=self.batch_size,
            )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            dataset=self.dataset_test_wrap,
            batch_size=self.batch_size,
            )

In [6]:
# Note that for Transformer decoder target mask,
# the `length` parameter is the desired length of
# the target sequence.
# See docs: https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html#torch.nn.Transformer.forward
def create_attn_mask(length: int):
    """Generate mask used for attention mechanisms.

    Masks are a lower-triangular matrix of zeros
    with the other entries taking value "-inf".

    Args:
        length (int): Length of square-matrix dimension.

    Examples:
        >>> create_attn_mask(3)
        tensor([[0., -inf, -inf],
                [0., 0., -inf],
                [0., 0., 0.]])
    """
    # Get lower-triangular matrix of ones.
    mask = torch.tril(torch.ones(length, length))

    # Replace 0 -> "-inf" and 1 -> 0.0
    mask = (
        mask
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask

In [7]:
class TimeseriesTransformer(torch.nn.Module):

    def __init__(self,
        n_input_features: int,
        n_output_features: int,
        d_model: int = 512,
        dropout: float = 0.1,
        batch_first: bool = False,
        n_encoder_layers: int = 4,
        n_decoder_layers: int = 4,
        ):
        super().__init__()

        self.batch_first = batch_first

        # Linear transformation from input-feature space into arbitrary n-dimension space.
        # This is similar to a word embedding used in NLP tasks.
        self.encoder_projection = torch.nn.Linear(in_features=n_input_features, out_features=d_model)
        self.decoder_projection = torch.nn.Linear(in_features=n_output_features, out_features=d_model)

        # Transformer encoder/decoder layers.
        encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=8, # Number of multihead-attention models.
            dropout=dropout,
            dim_feedforward=4*d_model,
            batch_first=batch_first,
        )
        decoder_layer = torch.nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=8, # Number of multihead-attention models.
            dropout=dropout,
            dim_feedforward=4*d_model,
            batch_first=batch_first,
        )
        self.encoder = torch.nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=n_encoder_layers)
        self.decoder = torch.nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=n_decoder_layers)

        # Linear output layer.
        # We typically only predict a single data point at a time, so output features is typically 1.
        self.linear = torch.nn.Linear(in_features=d_model, out_features=n_output_features)

    def encode(self, src: torch.Tensor) -> torch.Tensor:
        # Transform source into arbitrary feature space.
        x = self.encoder_projection(src)

        # # Create source mask.
        # if self.batch_first:
        #     src_length = src.size(1)
        # else:
        #     src_length = src.size(0)
        # src_mask = create_attn_mask(length=src_length).to(device=self.device)

        # Pass the linear transformation through the encoder layers.
        # x = self.encoder(x, mask=src_mask)
        x = self.encoder(x)

        return x

    def decode(self,
        tgt: torch.Tensor,
        memory: torch.Tensor,
        tgt_mask: torch.Tensor = None,
        ) -> torch.Tensor:
        """Decode function.

        Args:
            tgt (torch.Tensor): The sequence to the decoder
            memory (torch.Tensor): The sequence from the last layer of the encoder

        Returns:
            torch.Tensor: Decoded sequence.
        """
        # Transform target into arbitrary feature space.
        x = self.decoder_projection(tgt)

        # Pass the linear transformation through the decoder layers.
        x = self.decoder(tgt=x, memory=memory, tgt_mask=tgt_mask)

        # Pass the output of the decoder through the linear prediction layer.
        x = self.linear(x)
        return x

    def forward(self, 
        src: torch.Tensor,
        tgt: torch.Tensor,
        tgt_mask: torch.Tensor = None,
        ) -> torch.Tensor:
        x = self.encode(src)
        y = self.decode(tgt=tgt, memory=x, tgt_mask=tgt_mask)
        return y

In [8]:
class BeijingPM25ForecastTransformer(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.criterion = torch.nn.MSELoss(reduction='mean')

        # Create the transformer model.
        self.model = TimeseriesTransformer(*args, **kwargs)
        
        # Save parameters for checkpoint.
        self.save_hyperparameters()

    def forward(self, *args, **kwargs) -> torch.Tensor:
        return self.model(*args, **kwargs)

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.model.parameters(), 
            lr=1e-3, 
            betas=[0.9, 0.98], 
            eps=1e-9,
            )

    def compute_loss(self, y_hat, y):
        return self.criterion(y_hat, y)

    # Legacy method for reference.
    def training_step_old(self, batch, batch_idx):
        history_x, history_y, horizon_x, horizon_y = batch
        # y_hat = self((history_x, history_y,))
        y_hat = self((history_x, horizon_y,))
        loss = self.compute_loss(y_hat, horizon_y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def step(self, batch: torch.Tensor, batch_idx: int, key: str) -> float:
        """Generic step function used for train/validation/test loops.

        Args:
            batch (torch.Tensor): Tensor of batched records.
            batch_idx (int): Index of batch relative to entire dataset.
            key (str): Step key for logging purposes (one of ['train','val','test']).

        Returns:
            float: Prediction loss.
        """
        history_x, history_y, horizon_x, horizon_y = batch

        # Create decoder input sequence.
        # This should start with the last element of the encoder sequence
        # and end with the second-to-last element of the target sequence.
        tgt = torch.cat(
            (history_x[:,[-1],:], horizon_x[:,:-1,:]),
            dim=1
            )
        
        # Create target attention mask to prevent lookahead cheating.
        if self.model.batch_first:
            tgt_length = tgt.size(1)
        else:
            tgt_length = tgt.size(0)
        tgt_mask = create_attn_mask(length=tgt_length).to(device=self.device)

        # Pass source and target sequences into the transformer.
        y_hat = self(history_x, tgt, tgt_mask)

        # Compute loss.
        # loss = self.compute_loss(y_hat, horizon_y)
        loss = self.compute_loss(y_hat, horizon_x)
        self.log(f'{key}_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def training_step(self, batch: torch.Tensor, batch_idx: int) -> float:
        return self.step(batch, batch_idx, key='train')

    def test_step(self, batch: torch.Tensor, batch_idx: int):
        return self.step(batch, batch_idx, key='test')

In [9]:
# %reload_ext tensorboard
# %tensorboard --logdir ./lightning_logs

In [10]:
# Define parameters for the dataset.
feature_cols = [0,1,2,3]
target_cols = [-3]
history = 5
horizon = 3
split = 0.15
batch_size = 64

# Create the dataset.
dm = BeijingPM25LightningDataModule(
    root=dataset_root,
    feature_cols=feature_cols,
    target_cols=target_cols,
    history=history,
    horizon=horizon,
    split=split,
    batch_size=batch_size,
)

n_input_features: int = len(feature_cols)
# n_output_features: int = len(target_cols)
n_output_features: int = len(feature_cols)
d_model: int = 512
dropout: float = 0.2
model = BeijingPM25ForecastTransformer(
    n_input_features=n_input_features,
    n_output_features=n_output_features,
    d_model=d_model,
    dropout=dropout,
    batch_first=True,
    n_encoder_layers=4,
    n_decoder_layers=4,
    )

kwargs = {}
if torch.cuda.is_available():
    kwargs['gpus'] = 1

# Devlopment mode.
kwargs['fast_dev_run'] = True

trainer = pl.Trainer(**kwargs)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
/Users/derieux/My Drive/Virginia Tech/graduate/research/makassar/repos/makassar-ml/__pypackages__/3.9/lib/pytorch_lightning/trainer/configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")

  | Name      | Type                  | Params
----------------------------------------------------
0 | criterion | MSELoss               | 0     
1 | model     | TimeseriesTransformer | 29.4 M
----------------------------------------------------
29.4 M    Trainable params
0         Non-trainable params
29.4 M    Total params
117.731   Total estimated model params size (MB)
/Users/derieux/My Drive/Virginia Tech/graduate

Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  1.48s/it, loss=1.01e+06, v_num=, train_loss_step=1.01e+6, train_loss_epoch=1.01e+6]


## Testing

Here we try to forecast some features and plot them to see how well model does.

In [11]:
trainer.test(model, dm)

/Users/derieux/My Drive/Virginia Tech/graduate/research/makassar/repos/makassar-ml/__pypackages__/3.9/lib/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 100%|██████████| 1/1 [00:00<00:00,  2.28it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1003820.1875, 'test_loss_epoch': 1003820.1875}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 1/1 [00:00<00:00,  2.27it/s]


[{'test_loss': 1003820.1875, 'test_loss_epoch': 1003820.1875}]

In [12]:
# dm.prepare_data()
# dm.setup()
# train = dm.train_dataloader()
# val = dm.val_dataloader()
# test = dm.test_dataloader()

# # Print counts for each split.
# print('train',len(train))
# print('val',len(val))
# print('test',len(test))
# print('total',len(train)+len(val)+len(test))

# # Visually inspect the split boundaries to ensure that no values are missing.
# print('train[0]:',dm.dataset_train[0][0:4])
# print('train[-1]:',dm.dataset_train[-1][0:4])
# print('val[0]:',dm.dataset_val[0][0:4])
# print('val[-1]:',dm.dataset_val[-1][0:4])
# print('test[0]:',dm.dataset_test[0][0:4])
# print('test[-1]:',dm.dataset_test[-1][0:4])